![Redis](https://redis.io/wp-content/uploads/2024/04/Logotype.svg?auto=webp&quality=85,75&width=120)

# System Instructions: The Foundation of Context Engineering

## Learning Objectives (25 minutes)
By the end of this notebook, you will be able to:
1. **Build** effective system instructions step by step
2. **Test** different instruction approaches and see their impact
3. **Apply** behavioral guidelines to control agent responses
4. **Integrate** Redis University context into system prompts
5. **Optimize** instructions for specific use cases

## Prerequisites
- OpenAI API key configured
- Basic understanding of LLM prompting
- Redis Stack running (for advanced examples)

---

## Introduction

**System instructions** are the foundation of any AI agent. They define:
- **Who** the agent is (role and identity)
- **What** the agent can do (capabilities and tools)
- **How** the agent should behave (personality and guidelines)
- **When** to use different approaches (context-aware responses)

In this notebook, we'll build system instructions for our Redis University Class Agent step by step, testing each version to see the impact.

## Environment Setup

In [ ]:
# Environment setup
import os
from dotenv import load_dotenv

# Load environment variables
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

print("🔧 Environment Setup")
print("=" * 30)
print(f"OpenAI API Key: {'✅ Set' if OPENAI_API_KEY else '❌ Not set'}")

In [ ]:
# Import LangChain components (consistent with our LangGraph agent)
try:
    from langchain_openai import ChatOpenAI
    from langchain_core.messages import SystemMessage, HumanMessage, AIMessage
    
    if OPENAI_API_KEY:
        # Initialize LangChain ChatOpenAI (same as our agent uses)
        llm = ChatOpenAI(
            model="gpt-4o-mini",  # Using mini for cost-effective learning
            temperature=0.7,
            max_tokens=200  # Keep responses concise for learning
        )
        print("✅ LangChain ChatOpenAI initialized")
        print("🤖 Using gpt-4o-mini for cost-effective learning")
    else:
        llm = None
        print("⚠️  LangChain ChatOpenAI not available (API key not set)")
        print("You can still follow along with the examples!")
        
except ImportError as e:
    llm = None
    print(f"⚠️  LangChain not installed: {e}")
    print("Install with: pip install langchain-openai langchain-core")

## 🧪 Hands-on: Building System Instructions

Let's build system instructions for our Redis University Class Agent step by step. We'll start simple and add complexity gradually.

### Step 1: Minimal System Instructions

Let's start with the bare minimum and see what happens:

In [ ]:
# Minimal system prompt
minimal_prompt = "You are a helpful assistant."

print("📋 Minimal system prompt:")
print(minimal_prompt)
print("\n" + "="*50)

In [ ]:
# Test the minimal prompt using LangChain (consistent with our agent)
def test_prompt(system_prompt, user_message, label):
    """Helper function to test prompts using LangChain messages"""
    if llm:
        # Create LangChain messages (same pattern as our agent)
        messages = [
            SystemMessage(content=system_prompt),
            HumanMessage(content=user_message)
        ]
        
        # Invoke the LLM (same as our agent does)
        response = llm.invoke(messages)
        
        print(f"🤖 {label}:")
        print(response.content)
    else:
        print(f"⚠️  {label}: LangChain LLM not available - skipping test")
    
    print("\n" + "="*80)

# Test with a course planning question
test_prompt(
    minimal_prompt, 
    "I need help planning my classes for next semester.",
    "Response with minimal instructions"
)

**🤔 Problem:** The agent doesn't know it's a class scheduling agent. It might give generic advice instead of using our course catalog and tools.

### Step 2: Adding Role and Purpose

Let's give the agent a specific role:

In [ ]:
# Add role and purpose
role_prompt = """You are the Redis University Class Agent.

Your role is to help students:
• Find courses that match their interests and requirements
• Plan their academic schedule
• Check prerequisites and eligibility
• Get personalized course recommendations
"""

print("📋 System prompt with role:")
print(role_prompt)

In [ ]:
# Test the role-based prompt
test_prompt(
    role_prompt,
    "I need help planning my classes for next semester.",
    "Response with role and purpose"
)

**✅ Better!** The agent now understands its role, but it still doesn't know about our tools or how to behave.

### Step 3: Adding Behavioral Guidelines

Let's add some personality and behavior guidelines:

In [ ]:
# Add behavioral guidelines
behavior_prompt = """You are the Redis University Class Agent.

Your role is to help students:
• Find courses that match their interests and requirements
• Plan their academic schedule
• Check prerequisites and eligibility
• Get personalized course recommendations

Guidelines:
• Be helpful, friendly, and encouraging
• Ask clarifying questions when needed
• Provide specific course recommendations with details
• Explain prerequisites and requirements clearly
• Stay focused on course planning and scheduling
• If asked about topics outside your domain, politely redirect to course planning
"""

print("📋 System prompt with behavior guidelines:")
print(behavior_prompt)

In [ ]:
# Test with an off-topic question
test_prompt(
    behavior_prompt,
    "What's the weather like today?",
    "Response to off-topic question"
)

**🎯 Great!** The agent now stays focused on its purpose and redirects off-topic questions.

### Step 4: Adding Tool Awareness

Let's tell the agent about its capabilities and tools:

In [ ]:
# Add tool awareness
tools_prompt = """You are the Redis University Class Agent.

Your role is to help students:
• Find courses that match their interests and requirements
• Plan their academic schedule
• Check prerequisites and eligibility
• Get personalized course recommendations

You have access to:
• Complete course catalog with real-time availability
• Student academic records and transcripts
• Prerequisite checking tools
• Course recommendation engine
• Schedule conflict detection

Guidelines:
• Be helpful, friendly, and encouraging
• Ask clarifying questions when needed
• Use your tools to provide accurate, up-to-date information
• Explain prerequisites and requirements clearly
• Stay focused on course planning and scheduling
• If asked about topics outside your domain, politely redirect to course planning
"""

print("📋 System prompt with tool awareness:")
print(tools_prompt)

In [ ]:
# Test with a specific course question
test_prompt(
    tools_prompt,
    "I'm interested in machine learning courses. What do you recommend?",
    "Response with tool awareness"
)

**🚀 Excellent!** The agent now mentions using its tools and provides more specific guidance.

### Step 5: Adding Context and Examples

Let's add some context about Redis University and example interactions:

In [ ]:
# Add context and examples
complete_prompt = """You are the Redis University Class Agent.

Your role is to help students:
• Find courses that match their interests and requirements
• Plan their academic schedule
• Check prerequisites and eligibility
• Get personalized course recommendations

You have access to:
• Complete course catalog with real-time availability
• Student academic records and transcripts
• Prerequisite checking tools
• Course recommendation engine
• Schedule conflict detection

About Redis University:
• Focuses on data engineering, databases, and distributed systems
• Offers courses from beginner to advanced levels
• Supports both online and in-person learning formats
• Emphasizes hands-on, practical learning

Guidelines:
• Be helpful, friendly, and encouraging
• Ask clarifying questions when needed
• Use your tools to provide accurate, up-to-date information
• Explain prerequisites and requirements clearly
• Stay focused on course planning and scheduling
• If asked about topics outside your domain, politely redirect to course planning

Example interactions:
Student: "I'm new to databases. Where should I start?"
You: "Great question! For database beginners, I'd recommend starting with 'Introduction to Databases' (DB101). Let me check if you meet the prerequisites and find the best schedule for you."
"""

print("📋 Complete system prompt:")
print(complete_prompt)

In [ ]:
# Test the complete prompt
test_prompt(
    complete_prompt,
    "I'm new to databases. Where should I start?",
    "Response with complete instructions"
)

**🎉 Perfect!** The agent now has complete context and responds appropriately to Redis University students.

## 🔄 Hands-on: Comparing All Versions

Let's test all our versions side by side to see the evolution:

In [ ]:
# Compare all versions with the same question
test_question = "I want to learn about Redis and databases. What courses should I take?"

print("🔍 Testing all versions with the same question:")
print(f"Question: {test_question}")
print("\n" + "="*100)

# Test each version
versions = [
    ("Minimal", minimal_prompt),
    ("With Role", role_prompt),
    ("With Behavior", behavior_prompt),
    ("With Tools", tools_prompt),
    ("Complete", complete_prompt)
]

for name, prompt in versions:
    test_prompt(prompt, test_question, f"{name} Version")

## 🎮 Try It Yourself: Experiment with Instructions

Now that you understand the basics, try these experiments:

### Experiment 1: Change the Personality

Try making the agent more formal or more casual:

In [ ]:
# Formal version
formal_prompt = complete_prompt.replace(
    "Be helpful, friendly, and encouraging",
    "Be professional, precise, and academically rigorous"
)

# Casual version
casual_prompt = complete_prompt.replace(
    "Be helpful, friendly, and encouraging",
    "Be casual, enthusiastic, and use modern slang when appropriate"
)

print("🎭 Testing different personalities:")
test_question = "I'm struggling to choose between CS101 and CS102."

test_prompt(formal_prompt, test_question, "Formal Personality")
test_prompt(casual_prompt, test_question, "Casual Personality")

### Experiment 2: Add Constraints

Try adding specific constraints to see how they affect behavior:

In [ ]:
# Add constraints
constrained_prompt = complete_prompt + """

Important Constraints:
• Always ask for the student's year (freshman, sophomore, junior, senior) before recommending courses
• Never recommend more than 3 courses at once
• Always mention the time commitment for each course
• If a student seems overwhelmed, suggest starting with just one course
"""

print("🚧 Testing with constraints:")
test_prompt(
    constrained_prompt,
    "I want to learn everything about databases!",
    "Response with constraints"
)

### Experiment 3: Your Turn!

Create your own version with different characteristics:

In [ ]:
# Your custom prompt - modify this!
your_prompt = """You are the Redis University Class Agent.

Your role is to help students:
• Find courses that match their interests and requirements
• Plan their academic schedule
• Check prerequisites and eligibility
• Get personalized course recommendations

# TODO: Add your own personality, constraints, or special features here!
# Ideas:
# - Make it focus on career outcomes
# - Add industry connections
# - Include study tips
# - Add motivational elements
# - Focus on practical skills

Guidelines:
• Be helpful, friendly, and encouraging
• Ask clarifying questions when needed
• Use your tools to provide accurate, up-to-date information
"""

print("🎨 Your custom prompt:")
print(your_prompt)

# Test your custom prompt
test_prompt(
    your_prompt,
    "I want to get a job at a tech company. What courses should I prioritize?",
    "Your Custom Version"
)

## 🧪 Advanced: Simple Context Integration

Let's see how to make system instructions context-aware:

In [ ]:
# Simple context-aware prompt builder
def build_context_aware_prompt(student_info=None):
    """Build a prompt that includes student context"""
    
    base_prompt = complete_prompt
    
    if student_info:
        context_section = "\n\nCurrent Student Context:\n"
        
        if student_info.get('name'):
            context_section += f"• Student: {student_info['name']}\n"
        
        if student_info.get('year'):
            context_section += f"• Academic Year: {student_info['year']}\n"
        
        if student_info.get('major'):
            context_section += f"• Major: {student_info['major']}\n"
        
        if student_info.get('interests'):
            context_section += f"• Interests: {', '.join(student_info['interests'])}\n"
        
        context_section += "\nUse this context to personalize your recommendations.\n"
        
        return base_prompt + context_section
    
    return base_prompt

# Test with student context
student_context = {
    'name': 'Alice',
    'year': 'sophomore',
    'major': 'Computer Science',
    'interests': ['machine learning', 'web development']
}

context_prompt = build_context_aware_prompt(student_context)

print("🧠 Context-aware prompt:")
print(context_prompt[-200:])  # Show last 200 characters
print("\n" + "="*50)

In [ ]:
# Test context-aware response using LangChain
def test_context_aware_prompt(system_prompt, user_message, student_context):
    """Test context-aware prompts with student information"""
    if llm:
        # Build context-aware system message
        context_prompt = build_context_aware_prompt(student_context)
        
        # Create LangChain messages with context
        messages = [
            SystemMessage(content=context_prompt),
            HumanMessage(content=user_message)
        ]
        
        # Invoke with context (same pattern as our agent)
        response = llm.invoke(messages)
        
        print("🤖 Context-Aware Response:")
        print(f"Student: {student_context.get('name', 'Unknown')} ({student_context.get('year', 'Unknown')})")
        print(f"Interests: {', '.join(student_context.get('interests', []))}")
        print("\nResponse:")
        print(response.content)
    else:
        print("⚠️  Context-aware test: LangChain LLM not available")
    
    print("\n" + "="*80)

# Test with student context
test_context_aware_prompt(
    complete_prompt,
    "What courses should I take next semester?",
    student_context
)

## 🎯 Key Takeaways

From this hands-on exploration, you've learned:

### ✅ **System Instruction Fundamentals**
- **Start simple** and build complexity gradually
- **Test each change** to see its impact immediately
- **Role and purpose** are the foundation of effective instructions
- **Behavioral guidelines** control how the agent responds

### ✅ **Practical Techniques**
- **Progressive building** from minimal to complete instructions
- **Comparative testing** to see the evolution of responses
- **Constraint addition** to control specific behaviors
- **Context integration** for personalized responses

### ✅ **Best Practices**
- **Be specific** about the agent's role and capabilities
- **Include examples** of desired interactions
- **Add constraints** to prevent unwanted behavior
- **Test thoroughly** with various types of questions

### 🚀 **Next Steps**
You're now ready to:
- Build effective system instructions for any AI agent
- Test and iterate on instruction effectiveness
- Integrate context for personalized experiences
- Move on to **Tool Definition** to give your agent capabilities

---

**Great work!** You've mastered the fundamentals of system instruction design. Ready to continue with **`02_defining_tools.ipynb`** to learn how to give your agent powerful capabilities?